In [13]:
import pandas as pd
import glob
import os
import re

In [21]:
!pip install ekonlpy

     ---------------------------------------- 0.0/33.2 MB ? eta -:--:--
     -- ------------------------------------- 2.1/33.2 MB 9.0 MB/s eta 0:00:04
     --------------- ----------------------- 12.8/33.2 MB 31.0 MB/s eta 0:00:01
     --------------------------- ----------- 23.1/33.2 MB 36.5 MB/s eta 0:00:01
     --------------------------------------  32.8/33.2 MB 38.5 MB/s eta 0:00:01
     ---------------------------------------- 33.2/33.2 MB 36.9 MB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/10.3 MB ? eta -:--:--
   -------------------------------------- - 10.0/10.3 MB 47.8 MB/s eta 0:00:01
   ---------------------------------------- 10.3/10.

  You can safely remove it manually.
  You can safely remove it manually.


In [18]:
import re

def clean_news(text):
    if not isinstance(text, str):
        return ''
    
    # HTML 태그 제거
    text = re.sub(r'<[^>]+>', '', text)
    
    # 기자 이메일, 언론사 이름 제거
    text = re.sub(r'\[.+?기자\]|\[.+?뉴스\]|\[.+?제공\]', '', text)
    text = re.sub(r'\/사진제공?=.+?$', '', text)
    text = re.sub(r'\/사진?=.+?$', '', text)
    
    # 홍보 문구 제거
    text = re.sub(r'이\s*기사는\s*.+?\s*에\s*게재된\s*기사입니다\.', '', text)

    # 줄바꿈 공백으로
    text = text.replace('\u2028', ' ').replace('\u2029', ' ').replace('\n', ' ')
    
    # 한글, 영어, 숫자, 공백, 마침표만 남기기 (마침표는 문장 구분을 위해 남겨야 함)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s\.]', ' ', text)
    
    # 연속된 공백 하나로 줄이기
    text = ' '.join(text.split())
    
    return text

In [20]:
# 여러 언론사 클렌징 데이터 합치기
from tqdm import tqdm
tqdm.pandas()

folder_path = '../../db/news_contents/news_contents_*.csv'
file_list = glob.glob(folder_path)

all_data = []

for file in file_list:
    filename = os.path.basename(file)
    source_name = filename.split('_')[-1].replace('.csv', '')

    df = pd.read_csv(file, usecols=['date', 'content'])

    # 오염 데이터 제거용
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df = df.dropna(subset=['date'])

    temp_df = pd.DataFrame()
    temp_df['date'] = pd.to_datetime(df['date'])
    temp_df['date'] = temp_df['date'].dt.date

    temp_df['content'] = df['content'].progress_apply(clean_news)
    temp_df['tokens'] = ''
    temp_df['category'] = '뉴스'
    temp_df['source'] = source_name

    all_data.append(temp_df)
    print(f"✔ {source_name} 처리 완료 ({len(temp_df)}건)")

if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    final_df = final_df[['date', 'content', 'tokens', 'category', 'source']]
    final_df.to_csv('news_preprocessed_integrated.csv', index=False, encoding='utf-8-sig')
    print(f"\n✨ 총 {len(final_df)}건의 '뉴스' 데이터 저장 완료")


100%|██████████| 71236/71236 [00:03<00:00, 21872.43it/s]


✔ 매일경제 처리 완료 (71236건)


100%|██████████| 89012/89012 [00:05<00:00, 17015.67it/s]


✔ 머니투데이 처리 완료 (89012건)


100%|██████████| 79208/79208 [00:04<00:00, 19770.41it/s]


✔ 한국경제 처리 완료 (79208건)

✨ 총 239456건의 '뉴스' 데이터 저장 완료


In [22]:
from ekonlpy.sentiment import MPCK

mpck = MPCK()

def get_tokens(text):
    try:
        if not text:
            return ""
        tokens = mpck.tokenize(text)
        ngrams = mpck.ngramize(tokens)
        final_result = tokens + ngrams    
        return ", ".join(final_result)
    except:
        return ""

In [16]:
from tqdm import tqdm
print("--- 20만 건 토큰화 시작 (시간이 다소 소요됩니다) ---")
tqdm.pandas()
final_df['tokens'] = final_df['content'].progress_apply(get_tokens)

final_df.to_csv('news_preprocessed_with_tokens.csv', index=False, encoding='utf-8-sig')

--- 20만 건 토큰화 시작 (시간이 다소 소요됩니다) ---


100%|██████████| 300/300 [00:03<00:00, 82.99it/s] 


In [17]:
final_df

,date,content,tokens,category,source
0,2025-11-14,14일 서울 중구 하나은행 본점 딜링룸 현황판에 코스피가 표시돼 있다. 나흘 연속 ...,"중구/NNG, 본점/NNG, 딜링룸/NNG, 현황/NNG, 판/NNG, 코스피/NN...",뉴스,매일경제
1,2025-10-21,NH농협은행이 수탁 중인 전국 광역자치단체 금고의 예치액에 대한 금리가 최대 1.2...,"수탁/NNG, 중/NNG, 전국/NNG, 금고/NNG, 예치/NNG, 액/NNG, ...",뉴스,매일경제
2,2025-12-18,유럽중앙은행 ECB 이 18일 현지시간 예금금리를 비롯한 3대 정책금리를 모두 동결...,"ecb/NNG, ecb/NNG, 현지/NNG, 예금/NNG, 금리/NNG, 비롯/N...",뉴스,매일경제
3,2025-12-14,은행채 금리상승에 조달비용 대형증권사 IMA 출시도 변수 증권사 머니무브 차단 목적...,"은행채/NNG, 금리/NNG, 상승/NNG, 조달/NNG, 비용/NNG, 출시/NN...",뉴스,매일경제
4,2025-11-22,비트코인 급락에 스트레티지도 60 폭락 MSCI 나스닥100등 벤치마크 제외가능성 ...,"비트코인/NNG, 급락/NNG, 하락/NNG, msci/NNG, 나스닥100/NNG...",뉴스,매일경제
...,...,...,...,...,...
295,2025-12-11,미 기준금리 0.25 포인트 인하 환율 하락 출발 제롬 파월 미 중앙은행 Fed 의...,"미/NNG, 금리/NNG, 포인트/NNG, 인하/NNG, 환율/NNG, 하락/NNG...",뉴스,한국경제
296,2025-12-04,사진 연합뉴스 간밤 뉴욕증시가 기술주 없는 상승 을 선보였다. 부진한 고용지표를 바...,"간밤/NNG, 없/VA, 상승/NNG, 선보였/VV, 부진/NNG, 고용/NNG, ...",뉴스,한국경제
297,2025-12-11,Fed 3연속 금리 인하 위원 3명 반대 이례적 2명 동결 1명 빅컷 주장 3명 반...,"fed/NNG, 연속/NNG, 금리/NNG, 인하/NNG, 반대/NNG, 이례적/V...",뉴스,한국경제
298,2025-12-15,사진 뉴스1 국내 증시는 이번주 반도체 풍향계 마이크론 실적 발표를 앞두고 변동성을...,"뉴스/NNG, 증시/NNG, 주/NNG, 풍향계/NNG, 실적발표/NNG, 앞두/V...",뉴스,한국경제
